# Bibliotecas e Ambiente

In [ ]:
import warnings
from datetime import datetime
import numpy as np
import pandas as pd
from capymoa.classifier import (
    LeveragingBagging,
    HoeffdingTree,
    HoeffdingAdaptiveTree,
    AdaptiveRandomForestClassifier,
)
from capymoa.evaluation import (
    ClassificationEvaluator,
    prequential_evaluation, 
    ClassificationWindowedEvaluator,
    prequential_evaluation_multiple_learners,
    prequential_ssl_evaluation
)
from capymoa.evaluation.visualization import plot_windowed_results
from capymoa.stream import NumpyStream
from sklearn.preprocessing import LabelEncoder
from py.functions import plot_confusion_matrix, criar_stream, remover_colunas_correlacionadas
import random
from collections import deque
from capymoa.drift.detectors import DDM, ADWIN, ABCD
from py.models import get_models

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import re
from scipy.stats import linregress

# Configuração de Avisos 
warnings.filterwarnings('ignore')

# Streams

# Desenvolvimento e Testes

## Cenário 1

## Cenário 2

## Cenário 3